In [77]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
import cvxpy as cp

## Group Assignment
### Team Number: 08
### Team Member Names:
### Team Strategy Chosen: Market Meet

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


In [78]:
# tickers_file_name = "Tickers_Example.csv"
# tickers_file_original = pd.read_csv(tickers_file_path , header = None, names=['Ticker'])
tickers_file_path = r"c:\Users\abbas\OneDrive - University of Waterloo\UWaterloo 1A Fall 2024\CFM 101\Assignments\Group Assignment\Tickers_ExampleNew.csv"
tickers_file_original = pd.read_csv(tickers_file_path , header = None, names=['Ticker'])

start_date = "2023-10-01"
end_date = "2024-09-30"
drop_tickers = []

for tick in tickers_file_original['Ticker']:    

    try:

        ticker = yf.Ticker(tick) 

        if ticker.info['quoteType'] != 'EQUITY':
            drop_tickers.append(tick)
            continue
                
        if ticker.fast_info.get('currency') not in ['USD', 'CAD']:
            drop_tickers.append(tick)
            continue

        history = ticker.history(start = start_date, end = end_date)

        trading_days = history['Volume'].resample('ME').count()
        volume = history['Volume'].resample('ME').sum()
        trading_days = trading_days[trading_days>=18]
        volume = volume[volume.index==trading_days.index]
        monthly_volume = volume.mean()

        if monthly_volume<100000: 
            drop_tickers.append(tick)

    except:
            drop_tickers.append(tick)
            pass
    
tickers_file_original.drop(tickers_file_original[tickers_file_original["Ticker"].isin(drop_tickers)].index, inplace=True)
tickers_file_original = tickers_file_original.drop_duplicates(subset=['Ticker'], keep='last')
tickers_file_original = tickers_file_original.reset_index(drop=True)
tickers_file_original.index = tickers_file_original.index + 1

$NVAC: possibly delisted; no price data found  (period=5d)


In [79]:
# take exchange rates from yfinance
exchange = 'USDCAD=X'
exchange_ticker = yf.Ticker(exchange)
exchange_rate = exchange_ticker.history(interval = '1mo', start=start_date, end=end_date)["Close"]
exchange_rate = pd.DataFrame(exchange_rate)
exchange_rate.index = exchange_rate.index.date
exchange_rate_value = exchange_rate.mean().iloc[0]

In [80]:
sp500 = yf.Ticker("^GSPC").history(start=start_date, end=end_date)['Close'].mul(exchange_rate_value)
sp500.index = sp500.index.date
sp500_return = sp500.pct_change().dropna()


tsx60 = yf.Ticker("XIU.TO").history(start=start_date, end=end_date)['Close']
tsx60.index = tsx60.index.date
tsx60_return = tsx60.pct_change().dropna()

total_investment = 1000000

sp500val = (total_investment / sp500.iloc[0]) * sp500.iloc[-1]
sp500val = round(sp500val,2)

tsx60val = (total_investment / tsx60.iloc[0]) * tsx60.iloc[-1]
tsx60val = round(tsx60val,2)

average = (sp500val + tsx60val) / 2
average = round(average,2)

common_dates = sp500_return.index.intersection(tsx60_return.index)

average_df = pd.DataFrame(data = {"% Return" : (sp500_return.loc[common_dates] + tsx60_return.loc[common_dates])/2}, index = common_dates)


In [81]:
stocks = {key: 0 for key in tickers_file_original['Ticker'].values}
current_data = pd.DataFrame()
metrics = pd.DataFrame(index=list(stocks.keys()), columns=['SP-beta','SP-corr','TSX-beta','TSX-corr', 'S&P500', 'TSX60'])

url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_df = pd.read_html(url)[0]
sp500companies = sp500_df.Symbol.tolist()

tsx60companies = ["RY.TO", "TD.TO", "SHOP.TO", "ENB.TO", "CNR.TO", "SU.TO", "BN.TO", "CSU.TO", 
    "MFC.TO", "TRP.TO", "BMO.TO", "BNS.TO", "ATD.TO", "ABX.TO", "BCE.TO", "CM.TO", 
    "CP.TO", "NTR.TO", "TRI.TO", "WCN.TO", "MG.TO", "L.TO", "IFC.TO", "WN.TO", 
    "SLF.TO", "FNV.TO", "TECK.B.TO", "DOL.TO", "OTEX.TO", "FM.TO", "IMO.TO", 
    "CVE.TO", "PPL.TO", "CNQ.TO", "FTS.TO", "H.TO", "RCI.B.TO", "T.TO", "AQN.TO", 
    "SAP.TO", "MRU.TO", "QSR.TO", "WPM.TO", "POW.TO", "EMA.TO", "AEM.TO", 
    "GIL.TO", "GIB.A.TO", "TOU.TO", "BEP.UN.TO", "BIP.UN.TO", "CTC.A.TO", 
    "CCL.B.TO", "K.TO", "WSP.TO", "IFP.TO"]

def in_sp500(ticker, sp500companies):
    if ticker in sp500companies:
        return 0.5
    else:
        return 0
    
def in_tsx60(ticker, tsx60companies):
    if ticker in tsx60companies:
        return 0.5
    else:
        return 0

    
sp500_returns = sp500.pct_change(fill_method=None)
tsx60_returns = tsx60.pct_change(fill_method=None)

for i in stocks:
    
    current_data[i] = yf.Ticker(i).history(start=start_date, end=end_date)["Close"]
    metrics.loc[i, 'S&P500'] = in_sp500(i, sp500companies)
    metrics.loc[i, 'TSX60'] = in_tsx60(i, tsx60companies)
    
    stock_returns = current_data[i].pct_change().dropna()
    stock_returns.index = stock_returns.index.date
    
    common_dates = stock_returns.index.intersection(sp500_returns.index).intersection(tsx60_returns.index)

    stock_returns = stock_returns.loc[common_dates]
    aligned_sp500_returns = sp500_returns.loc[common_dates]
    aligned_tsx60_returns = tsx60_returns.loc[common_dates]
    
    covariance = np.cov(stock_returns, aligned_sp500_returns)[0, 1]
    variance = np.var(aligned_sp500_returns)
    beta_sp500 = covariance / variance
    
    correlation_sp500 = np.corrcoef(stock_returns, aligned_sp500_returns)[0, 1]
    
    covariance_tsx = np.cov(stock_returns, aligned_tsx60_returns)[0, 1]
    variance_tsx = np.var(aligned_tsx60_returns)
    beta_tsx60 = covariance_tsx / variance_tsx
    
    correlation_tsx60 = np.corrcoef(stock_returns, aligned_tsx60_returns)[0, 1]
    
    metrics.loc[i, 'SP-corr'] = correlation_sp500
    metrics.loc[i, 'SP-beta'] = beta_sp500
    metrics.loc[i, 'TSX-corr'] = correlation_tsx60
    metrics.loc[i, 'TSX-beta'] = beta_tsx60

    stocks[i] = round(100 * (correlation_sp500 + beta_sp500 + beta_tsx60 + correlation_tsx60 + metrics.loc[i, 'S&P500'] + metrics.loc[i, 'TSX60']),2)



C:\Users\abbas\AppData\Local\Temp\ipykernel_45960\4076825213.py:40: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  stock_returns = current_data[i].pct_change().dropna()
C:\Users\abbas\AppData\Local\Temp\ipykernel_45960\4076825213.py:40: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  stock_returns = current_data[i].pct_change().dropna()
C:\Users\abbas\AppData\Local\Temp\ipykernel_45960\4076825213.py:40: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or speci

In [83]:
stocks = dict(sorted(stocks.items(), key=lambda item: item[1], reverse=True))
stocks = list(stocks.items())[:24]
stocks = dict(stocks)

In [84]:
def give_weight(num_stocks):
    stocks_now = list(stocks.keys())[:num_stocks]
    prices = np.array(current_data[stocks_now].iloc[0])
    last_values = np.array(current_data[stocks_now].iloc[-1])
    w = cp.Variable(num_stocks)
    my_value = cp.sum(cp.multiply(last_values, w * 1000000 / prices))
    min_weight = (100 / (2 * num_stocks)) / 100
    objective = cp.Minimize(my_value - sp500val)
    constraints = [
        cp.sum(w) == 1,           
        w >= min_weight,                    
        w <= 0.15                      
    ]
    problem = cp.Problem(objective, constraints)
    problem.solve()
    return w.value

In [85]:
def get_value(weights,i):
    last_row = current_data[list(stocks.keys())[:i]].iloc[-1]
    first_row = current_data[list(stocks.keys())[:i]].iloc[0]
    return np.sum(np.multiply(last_row,np.divide(weights*1000000,first_row)))

In [86]:
def stocks_ith_portfolio(numStocks):
    weight_now = give_weight(numStocks)
    curr_val = get_value(weight_now, numStocks)
    return curr_val, weight_now

In [87]:
diff = average
close = 0
best_weights = []
best = 0
for i in range(12,25):
    curr_val, weight_now = stocks_ith_portfolio(i)
    if (abs(average - curr_val) < diff):
        diff = abs(curr_val - average)
        best = i
        best_weights = weight_now
        close = curr_val

In [92]:
ranked_stocks = stocks

stockslist = list(ranked_stocks.keys())
weightlist = best_weights
weights = dict(zip(stockslist, weightlist))
total_money = 1000000
final_allocation = {}
for stock, weight in weights.items():
    if weight > 0:
        allocation = total_money * weight  
        ticker = yf.Ticker(stock)
        
        if ticker.fast_info['currency'] == 'USD':
            price = ticker.history(start="2024-11-22", end="2024-11-23")['Close'].iloc[0] * exchange_rate_value         
        else:
            price = ticker.history(start="2024-11-22", end="2024-11-23")['Close'].iloc[0]          

        num_shares = (allocation / price)
        # considering transaction costs
        if num_shares > 3950:
            num_shares = (allocation - 3.95)/price
            transaction_cost = 3.95
        else:
            num_shares = allocation/(price+0.001)
            transaction_cost = num_shares*0.001
        total_cost = num_shares * price
        
        final_allocation[stock] = {
            "Number of Shares": num_shares,
            "Transaction Cost": transaction_cost,
            "Total Money Spent": total_cost + transaction_cost,
            "Cumulative Transaction Cost": np.round(sum(stock["Transaction Cost"] for stock in final_allocation.values()), 2),
            "Cumulative Total": np.round(total_cost + transaction_cost+ sum(stock["Total Money Spent"] for stock in final_allocation.values()), 2)
        }
final = pd.DataFrame(final_allocation)

In [93]:
# put everything into the final portfolio

Portfolio_Final = pd.DataFrame({"Tickers": [], 
                   "Price":[], 
                   "Currency":[], 
                   "Shares":[], 
                   "Value":[], 
                   "Weight":[]})
cumulative_value = 0
cumulative_transaction_cost = 0
for stock, weight in weights.items():
    if weight > 0:
        allocation = total_money * weight  
        ticker = yf.Ticker(stock)
        if ticker.fast_info.get('currency') == 'USD':
            price = ticker.history(start="2024-11-22", end="2024-11-23")['Close'].iloc[0] * exchange_rate_value         
        else:
            price = ticker.history(start="2024-11-22", end="2024-11-23")['Close'].iloc[0]          
            
        num_shares = (allocation / price)
        
        # considering transaction costs
        if num_shares > 3950:
            num_shares = (allocation - 3.95)/price
            transaction_cost = 3.95
        else:
            num_shares = allocation/(price+0.001)
            transaction_cost = num_shares*0.001
        total_cost = num_shares * price

        value = price * num_shares
        cumulative_transaction_cost += transaction_cost

        new_row = {"Tickers": stock, 
                   "Price": price, 
                   "Currency": "CAD", 
                   "Shares": num_shares, 
                   "Value": value, 
                   "Weight": weight}
        Portfolio_Final.loc[len(Portfolio_Final)] = new_row 

Portfolio_Final.index = range(1, len(Portfolio_Final) + 1)

In [94]:
print(f"The Total Value of the Portfolio is {Portfolio_Final['Value'].sum(axis=0)}")
print(f"The Total Transaction Costs of the Portfolio is {cumulative_transaction_cost}")
print(f"The Sum of the weights of each stocks adds up to {(Portfolio_Final['Weight'].sum(axis=0) * 100):,.2f} %")

The Total Value of the Portfolio is 999990.7334816494
The Total Transaction Costs of the Portfolio is 9.266518349764539
The Sum of the weights of each stocks adds up to 100.00 %


In [95]:
Portfolio_Final

,Tickers,Price,Currency,Shares,Value,Weight
1,SHOP.TO,149.479996,CAD,1003.472005,149998.990966,0.150000
2,QCOM,213.091190,CAD,195.533523,41666.471128,0.041667
3,CAT,540.223348,CAD,77.128450,41666.589602,0.041667
4,TXN,269.099161,CAD,557.413267,149999.442589,0.150000
5,PYPL,117.927946,CAD,1271.952350,149998.728061,0.150000
6,USB,71.338462,CAD,584.061973,41666.082695,0.041667
7,AMZN,267.903158,CAD,404.373487,108332.934241,0.108333
8,C,94.918608,CAD,438.967959,41666.227680,0.041667
9,BLK,1408.638919,CAD,29.579360,41666.637149,0.041667
10,AXP,409.492798,CAD,101.751643,41666.564899,0.041667


In [96]:
Stocks_Final = pd.DataFrame({"Tickers": Portfolio_Final['Tickers'], 
                             "Shares": Portfolio_Final['Shares']})

Stocks_Final.to_csv('Stocks_Group_08.csv', index=False)


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Abbas, Gurmehar, Elaine